In [13]:
import json
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [14]:
vocab_size = 10000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

In [16]:
with open("Sarcasm_Headlines_Dataset.json", 'r') as f:
    datastore = [json.loads(line) for line in open('Sarcasm_Headlines_Dataset.json', 'r')]

sentences = []
labels = []
urls=[]

for item in datastore:
    sentences.append(item['headline'])
    labels.append(item['is_sarcastic'])
    urls.append(item['article_link'])

In [17]:
training_sentences = sentences[0:training_size]
testing_sentences = sentences[training_size:]
training_labels = labels[0:training_size]
testing_labels = labels[training_size:]

In [18]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [19]:
import numpy as np
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
testing_labels = np.array(testing_labels)

In [20]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 24)                408       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 25        
Total params: 160,433
Trainable params: 160,433
Non-trainable params: 0
_________________________________________________________________


In [22]:
num_epochs = 30
history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

Epoch 1/30
625/625 - 2s - loss: 0.6763 - accuracy: 0.5621 - val_loss: 0.6336 - val_accuracy: 0.7296
Epoch 2/30
625/625 - 1s - loss: 0.4703 - accuracy: 0.8053 - val_loss: 0.3931 - val_accuracy: 0.8331
Epoch 3/30
625/625 - 2s - loss: 0.3210 - accuracy: 0.8723 - val_loss: 0.3571 - val_accuracy: 0.8486
Epoch 4/30
625/625 - 1s - loss: 0.2673 - accuracy: 0.8932 - val_loss: 0.3571 - val_accuracy: 0.8465
Epoch 5/30
625/625 - 2s - loss: 0.2324 - accuracy: 0.9083 - val_loss: 0.3486 - val_accuracy: 0.8572
Epoch 6/30
625/625 - 2s - loss: 0.2040 - accuracy: 0.9201 - val_loss: 0.3515 - val_accuracy: 0.8520
Epoch 7/30
625/625 - 3s - loss: 0.1811 - accuracy: 0.9309 - val_loss: 0.3606 - val_accuracy: 0.8538
Epoch 8/30
625/625 - 2s - loss: 0.1610 - accuracy: 0.9402 - val_loss: 0.3733 - val_accuracy: 0.8529
Epoch 9/30
625/625 - 2s - loss: 0.1471 - accuracy: 0.9465 - val_loss: 0.3888 - val_accuracy: 0.8532
Epoch 10/30
625/625 - 2s - loss: 0.1326 - accuracy: 0.9517 - val_loss: 0.4149 - val_accuracy: 0.8484

In [23]:
sentence = ["granny starting to fear spiders in the garden might be real", "game of thrones season finale showing this sunday night"]
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))

[[9.8807442e-01]
 [3.6734343e-04]]
